# ETL da camada Silver para camada Gold

In [118]:
%pip install psycopg sqlalchemy pandas dotenv

Note: you may need to restart the kernel to use updated packages.


Esta célula importa todas as bibliotecas necessárias para o processo de ETL.
Aqui são carregados os pacotes para manipulação de dados (pandas), conexão com o banco de dados PostgreSQL (psycopg), controle de mensagens de erro (sys) e tratamento de avisos (warnings).
Ela deve ser executada antes de qualquer outra célula, pois fornece as dependências básicas que serão usadas nas etapas de Extract, Transform e Load.

In [119]:
import pandas as pd
import numpy as np
import psycopg
from psycopg import connect, sql
from psycopg2 import connect, sql
import psycopg2
import sys
import warnings
import math
from dotenv import load_dotenv
import os

warnings.filterwarnings('ignore')

# 1. Extract

Esta célula define as configurações de conexão com o banco de dados PostgreSQL e monta a consulta SQL que será usada para extrair os dados.
Ela cria variáveis com credenciais, monta o nome completo da tabela (schema.tabela) e gera a query SELECT * FROM DL.ORDER_ITEMS, além de preparar a connection string usada na etapa de conexão.

In [120]:
DB_SCHEMA = 'DL'
TABLE_NAME = 'ORDER_ITEMS'

TABLE_FULL_NAME = sql.SQL("{}.{}").format(
    sql.Identifier(DB_SCHEMA),
    sql.Identifier(TABLE_NAME)
)

def get_db_connection_info():
    load_dotenv()
    url = os.getenv('DB_URL')
    db_env = os.getenv('DB_ENV')
    if url is not None and db_env == 'prod':
        return url

    # credenciais do banco de dados local
    DB_USER = "postgres"
    DB_PASSWORD = "postgres"
    DB_HOST = "localhost"
    DB_PORT = "5433"
    DB_NAME = "olist"

    return f"host={DB_HOST} dbname={DB_NAME} user={DB_USER} password={DB_PASSWORD} port={DB_PORT}"


query_object = sql.SQL("SELECT * FROM {}").format(TABLE_FULL_NAME)

connection_string = get_db_connection_info()

Esta célula executa a extração dos dados do banco PostgreSQL.
Ela estabelece a conexão usando as configurações definidas anteriormente, converte o objeto SQL em uma query legível, executa a consulta e carrega o resultado no DataFrame df.
Em caso de falha na conexão ou na leitura, exibe uma mensagem de erro detalhada e encerra o processo.

In [121]:
DB_SCHEMA = 'DL'          
TABLE_NAME = 'ORDER_ITEMS' 

def get_db_connection_info():
    load_dotenv()
    url = os.getenv('DB_URL')
    db_env = os.getenv('DB_ENV')
    if url is not None and db_env == 'prod':
        return url

    # credenciais do banco de dados local
    DB_USER = "postgres"
    DB_PASSWORD = "postgres"
    DB_HOST = "localhost"
    DB_PORT = "5433"
    DB_NAME = "olist"

    return f"host={DB_HOST} dbname={DB_NAME} user={DB_USER} password={DB_PASSWORD} port={DB_PORT}"


connection_string = get_db_connection_info()

# Aqui montamos a query com schema + tabela de forma segura
query_object = sql.SQL("SELECT * FROM {}.{}").format(
    sql.Identifier(DB_SCHEMA),
    sql.Identifier(TABLE_NAME)
)

try:
    print("Estabelecendo conexão...")
    with connect(connection_string) as conn:
        print("Conexão estabelecida.")

        # Gera a string SQL final a partir do objeto seguro
        query_string = query_object.as_string(conn)
        print(f"Executando query: {query_string}")

        # Lê o resultado direto em um DataFrame
        df = pd.read_sql_query(query_string, conn)

    print("\nDados carregados do banco para o DataFrame com sucesso!")
except psycopg2.errors.UndefinedTable as e:
    print("\n--- A tabela informada não existe no banco de dados ---")
    print(f"Tente conferir o schema e o nome da tabela: {DB_SCHEMA}.{TABLE_NAME}")
    print(f"Erro detalhado: {e}")
    sys.exit(1)
except psycopg2.Error as e:
    print("\n--- Ocorreu um erro ao conectar ou ler o banco de dados ---")
    print(f"Erro: {e}")
    sys.exit(1)
except Exception as e:
    print("\n--- Ocorreu um erro inesperado ---")
    print(f"Erro: {e}")
    sys.exit(1)

Estabelecendo conexão...
Conexão estabelecida.
Executando query: SELECT * FROM "DL"."ORDER_ITEMS"

Dados carregados do banco para o DataFrame com sucesso!


Estas células exibem um resumo simples do resultado da extração, mostrando o número total de registros carregados no DataFrame df, as primeiras três tuplas e os tipos de cada dado.
Elas servem para confirmar visualmente que a consulta foi executada com sucesso e quantas linhas foram retornadas do banco.

In [122]:
print(f"Total de linhas carregadas: {len(df)}")

Total de linhas carregadas: 112952


In [123]:
df.head(3)

,order_item_id,product_id,seller_id,order_id,shipping_limit_date,price,freight_value,product_category_name,product_name_lenght,product_description_lenght,...,order_delivered_carrier_date,order_estimated_delivery_date,review_score,review_creation_date,review_answer_timestamp,customer_zip_code_prefix,customer_city,customer_state,cliente_geolocation_lat,cliente_geolocation_lng
0,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,00010242fe8c5a6d1ba2dd792cb16214,2017-09-19 09:45:35,58.9,13.29,cool_stuff,58.0,598.0,...,2017-09-19 18:34:16,2017-09-29,5.0,2017-09-21,2017-09-22 10:57:03,28013,campos dos goytacazes,RJ,-21.758076,-41.312633
1,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,00018f77f2f0320c557190d7a144bdd3,2017-05-03 11:05:13,239.9,19.93,pet_shop,56.0,239.0,...,2017-05-04 14:35:00,2017-05-15,4.0,2017-05-13,2017-05-15 11:34:13,15775,santa fe do sul,SP,-20.212393,-50.941471
2,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,000229ec398224ef6ca0657da4fc703e,2018-01-18 14:48:30,199.0,17.87,moveis_decoracao,59.0,695.0,...,2018-01-16 12:36:48,2018-02-05,5.0,2018-01-23,2018-01-23 16:06:31,35661,para de minas,MG,-19.860439,-44.597972


In [124]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112952 entries, 0 to 112951
Data columns (total 41 columns):
 #   Column                         Non-Null Count   Dtype         
---  ------                         --------------   -----         
 0   order_item_id                  112952 non-null  object        
 1   product_id                     112952 non-null  object        
 2   seller_id                      112952 non-null  object        
 3   order_id                       112952 non-null  object        
 4   shipping_limit_date            112952 non-null  datetime64[ns]
 5   price                          112952 non-null  float64       
 6   freight_value                  112952 non-null  float64       
 7   product_category_name          111342 non-null  object        
 8   product_name_lenght            111342 non-null  float64       
 9   product_description_lenght     111342 non-null  float64       
 10  product_photos_qty             111342 non-null  float64       
 11  

# 2. Transform

Esta célula define as colunas que farão parte de cada dimensão no modelo estrela.
Cada lista de colunas corresponde aos atributos que serão extraídos do DataFrame principal para formar as tabelas dimensionais DIM_PRODUTOS, DIM_VENDEDORES, DIM_PEDIDOS e a dimensão temporal DIM_DATA.

In [125]:
# Colunas para DIM_PRD
cols_produtos = [
    'product_category_name',
    'product_name_lenght',
    'product_description_lenght',
    'product_photos_qty',
    'product_weight_g',
    'product_length_cm',
    'product_height_cm',
    'product_width_cm'
]

# Colunas para DIM_VND
cols_vendedores = [
    'seller_zip_code_prefix',
    'seller_city',
    'seller_state',
    'vendedor_geolocation_lat',
    'vendedor_geolocation_lng'
]

# Colunas para DIM_ORD
cols_pedidos = [
    'rev_com_tit',
    'rev_com_msn',
    'cli_unq_id',
    'ord_stt',
    'qtd_pay_seq',
    'pri_pay_typ',
    'val_tot_pay',
    'max_pay_prc',
    'ord_pcs_ttp',
    'ord_env_cli_dat',
    'ord_apd_dat',
    'ord_env_pst_dat',
    'ord_est_env_dat',
    'rev_sco',
    'rev_cre_dat',
    'rev_ans_ttp',
    'cli_zip_cod_prf',
    'cli_cit',
    'cli_sta',
    'cliente_geolocation_lat',
    'cliente_geolocation_lng'
]

# Coluna para DIM_DAT (será extraída de ord_pcs_ttp)
col_data = 'ord_pcs_ttp'

Esta célula renomeia as colunas do DataFrame para corresponder à nomenclatura da camada Gold.
Os nomes são padronizados com prefixos mnemônicos (prod_, vend_, geo_) conforme definido no dicionário de dados da camada Gold.

In [126]:
# Renomear colunas para nomenclatura Gold
df = df.rename(columns={
    # Produtos
    'product_category_name': 'prd_cat_nam',
    'product_name_lenght': 'prd_nam_lgt',
    'product_description_lenght': 'prd_dsc_lgt',
    'product_photos_qty': 'prd_pic_qty',
    'product_weight_g': 'prd_wei_g',
    'product_length_cm': 'prd_lgt_cm',
    'product_height_cm': 'prd_hei_cm',
    'product_width_cm': 'prd_wid_cm',
    # Vendedores
    'seller_zip_code_prefix': 'vnd_zip_cod_prf',
    'seller_city': 'vnd_cit',
    'seller_state': 'vnd_sta',
    'vendedor_geolocation_lat': 'geo_lat',
    'vendedor_geolocation_lng': 'geo_lng',
    # Cliente (geolocalização)
    'cliente_geolocation_lat': 'cliente_geo_lat',
    'cliente_geolocation_lng': 'cliente_geo_lng',
    # Pedidos / Orders
    'order_purchase_timestamp': 'ord_pcs_ttp',
    'order_delivered_customer_date': 'ord_env_cli_dat',
    'order_approved_at': 'ord_apd_dat',
    'order_delivered_carrier_date': 'ord_env_pst_dat',
    'order_estimated_delivery_date': 'ord_est_env_dat',
    'review_comment_title': 'rev_com_tit',
    'review_comment_message': 'rev_com_msn',
    'customer_unique_id': 'cli_unq_id',
    'order_status': 'ord_stt',
    'qtd_payment_sequential': 'qtd_pay_seq',
    'primeiro_payment_type': 'pri_pay_typ',
    'valor_total_pagamento': 'val_tot_pay',
    'maximo_payment_installments': 'max_pay_prc',
    'review_score': 'rev_sco',
    'review_creation_date': 'rev_cre_dat',
    'review_answer_timestamp': 'rev_ans_ttp',
    'customer_zip_code_prefix': 'cli_zip_cod_prf',
    'customer_city': 'cli_cit',
    'customer_state': 'cli_sta',
    # Fato
    'shipping_limit_date': 'env_lmt_dat',
    'price': 'val',
    'freight_value': 'frt_val',
})

# Atualizar listas de colunas com novos nomes
cols_produtos = [
    'prd_cat_nam',
    'prd_nam_lgt',
    'prd_dsc_lgt',
    'prd_pic_qty',
    'prd_wei_g',
    'prd_lgt_cm',
    'prd_hei_cm',
    'prd_wid_cm'
]

cols_vendedores = [
    'vnd_zip_cod_prf',
    'vnd_cit',
    'vnd_sta',
    'geo_lat',
    'geo_lng',
]

cols_pedidos = [
    'rev_com_tit',
    'rev_com_msn',
    'cli_unq_id',
    'ord_stt',
    'qtd_pay_seq',
    'pri_pay_typ',
    'val_tot_pay',
    'max_pay_prc',
    'ord_pcs_ttp',
    'ord_env_cli_dat',
    'ord_apd_dat',
    'ord_env_pst_dat',
    'ord_est_env_dat',
    'rev_sco',
    'rev_cre_dat',
    'rev_ans_ttp',
    'cli_zip_cod_prf',
    'cli_cit',
    'cli_sta',
    'cliente_geo_lat',
    'cliente_geo_lng'
]

Esta célula cria campos calculados para a dimensão DIM_PEDIDOS.
Calcula o tempo de entrega em dias (tempo_entrega_dias) e a flag de atraso (flag_atraso) comparando a data de entrega real com a data estimada.

In [127]:
# Converter colunas de timestamp para datetime
df['ord_pcs_ttp'] = pd.to_datetime(df['ord_pcs_ttp'])
df['ord_env_cli_dat'] = pd.to_datetime(df['ord_env_cli_dat'])
df['ord_est_env_dat'] = pd.to_datetime(df['ord_est_env_dat'])

# Calcular tempo de entrega em dias
df['tem_de_env_dia'] = (df['ord_env_cli_dat'] - df['ord_pcs_ttp']).dt.days

# Calcular flag de atraso (1 se atrasou, 0 se não)
df['flg_atr'] = ((df['ord_env_cli_dat'] > df['ord_est_env_dat']).astype(int))

# Adicionar campos calculados à lista de colunas de pedidos
cols_pedidos.extend(['tem_de_env_dia', 'flg_atr'])

Esta célula cria a dimensão temporal DIM_DATA extraindo datas únicas do campo order_purchase_timestamp.
Para cada data única, extrai ano, mês, dia e dia da semana em português.

In [128]:
# Extrair datas únicas
df_data = df[[col_data]].copy()
df_data['dat_cmp'] = df_data[col_data].dt.date
df_data = df_data[['dat_cmp']].drop_duplicates().copy()

# Converter para datetime para extrair componentes
df_data['dat_cmp'] = pd.to_datetime(df_data['dat_cmp'])

# Extrair componentes da data
df_data['ano'] = df_data['dat_cmp'].dt.year
df_data['mes'] = df_data['dat_cmp'].dt.month
df_data['dia'] = df_data['dat_cmp'].dt.day

# Mapear dia da semana para português
dias_semana_pt = {
    0: 'Segunda',
    1: 'Terça',
    2: 'Quarta',
    3: 'Quinta',
    4: 'Sexta',
    5: 'Sábado',
    6: 'Domingo'
}
df_data['dia_da_sem'] = df_data['dat_cmp'].dt.dayofweek.map(dias_semana_pt)

# Converter dat_cmp de volta para date
df_data['dat_cmp'] = df_data['dat_cmp'].dt.date

Esta célula cria DataFrames separados para cada dimensão, removendo duplicatas.
Cada DataFrame conterá apenas as colunas relevantes para sua respectiva dimensão.

In [129]:
df_produtos = df[cols_produtos].drop_duplicates().copy()
df_vendedores = df[cols_vendedores].drop_duplicates().copy()
df_pedidos = df[cols_pedidos].drop_duplicates().copy()

Esta célula realiza a padronização e limpeza dos campos numéricos do DataFrame.
Ela converte todas as colunas de métricas para tipo numérico, substitui valores infinitos por NaN e depois transforma todos os NaN e valores ausentes em None, garantindo que o banco de dados receba NULL corretamente durante a carga.

In [130]:
num_cols = ['val', 'frt_val', 'tem_de_env_dia']
for c in num_cols:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors='coerce')

df = df.replace([np.inf, -np.inf], np.nan)
df = df.where(pd.notna(df), None)

Esta célula exibe um resumo da etapa de transformação, mostrando quantos registros foram preparados em cada dimensão e na tabela fato base.
Em seguida, utiliza df.info() para apresentar a estrutura geral do DataFrame principal, permitindo verificar tipos de dados e possíveis valores nulos antes da carga no banco.

In [131]:
print("Registros preparados para carga:")
print(f"DIM_PRD: {len(df_produtos)}")
print(f"DIM_VND: {len(df_vendedores)}")
print(f"DIM_ORD: {len(df_pedidos)}")
print(f"DIM_DAT: {len(df_data)}")
print(f"FAT_ITS_ORD (base df): {len(df)}\n\n")

df.info()

Registros preparados para carga:
DIM_PRD: 32256
DIM_VND: 2296
DIM_ORD: 98909
DIM_DAT: 616
FAT_ITS_ORD (base df): 112952


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112952 entries, 0 to 112951
Data columns (total 43 columns):
 #   Column           Non-Null Count   Dtype         
---  ------           --------------   -----         
 0   order_item_id    112952 non-null  object        
 1   product_id       112952 non-null  object        
 2   seller_id        112952 non-null  object        
 3   order_id         112952 non-null  object        
 4   env_lmt_dat      112952 non-null  datetime64[ns]
 5   val              112952 non-null  float64       
 6   frt_val          112952 non-null  float64       
 7   prd_cat_nam      111342 non-null  object        
 8   prd_nam_lgt      111342 non-null  float64       
 9   prd_dsc_lgt      111342 non-null  float64       
 10  prd_pic_qty      111342 non-null  float64       
 11  prd_wei_g        112934 non-null  float64       
 12  prd_lg

# 3. Load

Esta célula configura os parâmetros de conexão com o banco de dados do Data Warehouse (DW) e valida se todas as variáveis geradas na etapa de transformação estão disponíveis na memória.
Ela garante que o ambiente esteja pronto antes de iniciar a fase de carga, evitando erros por falta de dados ou variáveis necessárias.

In [132]:
DB_SCHEMA_GOLD = "DW"

connection_string = get_db_connection_info()

for v in ['df', 'df_produtos', 'df_vendedores', 'df_pedidos', 'df_data', 'cols_produtos', 'cols_vendedores', 'cols_pedidos', 'col_data']:
    if v not in globals():
        raise RuntimeError(f"Variável ausente: {v}")

Esta célula executa o script DDL responsável por criar ou recriar as tabelas do schema DW no banco de dados.
Ela lê o arquivo SQL que contém a definição das tabelas e executa o comando dentro de uma conexão com o PostgreSQL, preparando a estrutura necessária para receber os dados na etapa de carga.

In [133]:
try:
    ddl_gold = open('../data_layer/gold/DDL.sql').read()
except FileNotFoundError:
    print("Erro: Arquivo 'DDL.sql' não encontrado em '../data_layer/gold/DDL.sql'.")
    sys.exit(1)

with connect(connection_string) as conn:
    with conn.cursor() as cur:
        cur.execute(ddl_gold)

In [134]:
# Verificar schemas DL e dl (investigação do problema)
print("🔍 Investigando schemas DL e dl...")
with connect(connection_string) as conn:
    df_schemas = pd.read_sql_query("""
        SELECT 
            schema_name,
            COUNT(*) as num_tables
        FROM information_schema.schemata
        WHERE schema_name IN ('DL', 'dl')
        GROUP BY schema_name
        ORDER BY schema_name;
    """, conn)
    
    if len(df_schemas) > 0:
        print("⚠️  ATENÇÃO: Encontrados múltiplos schemas similares:")
        print(df_schemas.to_string(index=False))
        print("\n💡 Dica: PostgreSQL diferencia maiúsculas/minúsculas quando o nome está entre aspas.")
        print("   Considere consolidar em um único schema (preferencialmente 'DL').\n")
    else:
        print("✓ Schema DL verificado")


🔍 Investigando schemas DL e dl...
⚠️  ATENÇÃO: Encontrados múltiplos schemas similares:
schema_name  num_tables
         DL           1
         dl           1

💡 Dica: PostgreSQL diferencia maiúsculas/minúsculas quando o nome está entre aspas.
   Considere consolidar em um único schema (preferencialmente 'DL').



Esta célula realiza a carga da dimensão Produtos no schema DW.
Ela insere todos os registros do DataFrame df_produtos na tabela DIM_PRODUTOS e adiciona uma linha extra com valores nulos para representar o registro 'desconhecido', armazenando sua chave substituta (unknown_prod_key) para uso posterior na carga da tabela fato.

In [135]:
with connect(connection_string) as conn:
    df_all_tables = pd.read_sql_query("""
        SELECT table_schema, table_name
        FROM information_schema.tables
        WHERE table_schema NOT IN ('pg_catalog', 'information_schema')
        ORDER BY table_schema, table_name;
    """, conn)

df_all_tables


,table_schema,table_name
0,DL,ORDER_ITEMS
1,dl,order_items
2,dw,dim_dat
3,dw,dim_ord
4,dw,dim_prd
5,dw,dim_vnd
6,dw,fat_its_ord
7,public,DIMDATA
8,public,DIMPEDIDOS
9,public,DIMPRODUTOS


In [136]:
# Carrega dimensão produto no DW, incluindo linha "desconhecido"

# Defina as colunas do banco para inserção
cols_produtos = [
    "prd_cat_nam",
    "prd_nam_lgt",
    "prd_dsc_lgt",
    "prd_pic_qty",
    "prd_wei_g",
    "prd_lgt_cm",
    "prd_hei_cm",
    "prd_wid_cm",
]

# Crie um DataFrame apenas com as colunas de interesse, renomeando se necessário
rename_cols = {
    "prod_category_name": "prd_cat_nam",
    "prod_name_lenght": "prd_nam_lgt",
    "prod_desc_lenght": "prd_dsc_lgt",
    "prod_photos_qty": "prd_pic_qty",
    "prod_weight_g": "prd_wei_g",
    "prod_length_cm": "prd_lgt_cm",
    "prod_height_cm": "prd_hei_cm",
    "prod_width_cm": "prd_wid_cm"
}
# Caso df_produtos já esteja com nomes internos, apenas seleciona
if all(col in df_produtos.columns for col in cols_produtos):
    df_ins = df_produtos[cols_produtos].copy()
else:
    df_ins = df_produtos.rename(columns=rename_cols)[cols_produtos].copy()

# Converte para tipos numéricos onde precisar
int_cols = ["prd_nam_lgt", "prd_dsc_lgt", "prd_pic_qty"]
dec_cols = ["prd_wei_g", "prd_lgt_cm", "prd_hei_cm", "prd_wid_cm"]

for col in int_cols:
    df_ins[col] = pd.to_numeric(df_ins[col], errors="coerce")  # Não numérico vira NaN

for col in dec_cols:
    df_ins[col] = pd.to_numeric(df_ins[col], errors="coerce")

# Garante None ao invés de NaN para inserir no banco
df_ins = df_ins.astype(object)
df_ins = df_ins.where(df_ins.notna(), None)

rows = list(df_ins.itertuples(index=False, name=None))

with connect(connection_string) as conn:
    with conn.cursor() as cur:
        insert_query = sql.SQL(
            """
            INSERT INTO DW.DIM_PRD (
                prd_cat_nam, prd_nam_lgt, prd_dsc_lgt, prd_pic_qty,
                prd_wei_g, prd_lgt_cm, prd_hei_cm, prd_wid_cm
            ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
            """
        )
        cur.executemany(insert_query, rows)

        # Insere linha "desconhecida" explicitamente e recupera chave surrogate
        cur.execute(
            """
            INSERT INTO DW.DIM_PRD (
                prd_cat_nam, prd_nam_lgt, prd_dsc_lgt, prd_pic_qty,
                prd_wei_g, prd_lgt_cm, prd_hei_cm, prd_wid_cm
            ) VALUES (NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL)
            RETURNING "SRK_prd";
            """
        )
        unknown_prod_key = cur.fetchone()[0]

        conn.commit()

print("Chave surrogate do produto desconhecido:", unknown_prod_key)

Chave surrogate do produto desconhecido: 32257


Esta célula insere os dados da dimensão Vendedores no schema DW.
Ela carrega todos os registros do DataFrame df_vendedores na tabela DIM_VENDEDORES e adiciona um registro adicional com valores nulos para representar o vendedor 'desconhecido', salvando sua chave substituta (unknown_vend_key) para uso posterior na carga da tabela fato.

In [137]:
with connect(connection_string) as conn:
    with conn.cursor() as cur:
        insert_query = sql.SQL(
            """INSERT INTO DW.DIM_VND (
                vnd_zip_cod_prf, vnd_cit, vnd_sta, geo_lat, geo_lng
            ) VALUES (%s, %s, %s, %s, %s)"""
        )
        cur.executemany(insert_query, [tuple(x) for x in df_vendedores.to_numpy()])
        cur.execute(
            """INSERT INTO DW.DIM_VND (
                vnd_zip_cod_prf, vnd_cit, vnd_sta, geo_lat, geo_lng
            ) VALUES (NULL, NULL, NULL, NULL, NULL) RETURNING "SRK_vnd" """
        )
        unknown_vend_key = cur.fetchone()[0]

Esta célula carrega os dados da dimensão Data no schema DW.
Ela insere os registros do DataFrame df_data na tabela DIM_DATA e adiciona um registro com valores nulos para representar datas desconhecidas, salvando sua chave substituta (unknown_data_key) que será usada na inserção da tabela fato.

In [138]:
with connect(connection_string) as conn:
    with conn.cursor() as cur:
        insert_query = sql.SQL(
            """INSERT INTO DW.DIM_DAT (
                dat_cmp, ano, mes, dia, dia_da_sem
            ) VALUES (%s, %s, %s, %s, %s)"""
        )
        cur.executemany(insert_query, [tuple(x) for x in df_data.to_numpy()])
        cur.execute(
            """INSERT INTO DW.DIM_DAT (
                dat_cmp, ano, mes, dia, dia_da_sem
            ) VALUES (NULL, NULL, NULL, NULL, NULL) RETURNING "SRK_dat" """
        )
        unknown_data_key = cur.fetchone()[0]

Esta célula realiza a carga da dimensão Pedidos no schema DW.
Ela insere os registros do DataFrame df_pedidos na tabela DIM_PEDIDOS e adiciona um registro com valores nulos para representar pedidos ausentes, armazenando a chave substituta (unknown_ord_key) que será utilizada posteriormente na carga da tabela fato.

In [139]:
with connect(connection_string) as conn:
    with conn.cursor() as cur:
        insert_query = sql.SQL(
            """INSERT INTO DW.DIM_ORD (
                rev_com_tit, rev_com_msn, cli_unq_id, ord_stt,
                qtd_pay_seq, pri_pay_typ, val_tot_pay, max_pay_prc,
                ord_pcs_ttp, ord_env_cli_dat, tem_de_env_dia, flg_atr,
                ord_apd_dat, ord_env_pst_dat, ord_est_env_dat,
                rev_sco, rev_cre_dat, rev_ans_ttp,
                cli_zip_cod_prf, cli_cit, cli_sta, geo_lat, geo_lng
            ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"""
        )

        # ---------- PREPARAÇÃO DO DATAFRAME ----------
        df_pedidos_load = df_pedidos.copy()
        df_pedidos_load = df_pedidos_load.rename(columns={
            'cliente_geo_lat': 'geo_lat',
            'cliente_geo_lng': 'geo_lng'
        })

        cols_pedidos_load = [
            'rev_com_tit', 'rev_com_msn', 'cli_unq_id', 'ord_stt',
            'qtd_pay_seq', 'pri_pay_typ', 'val_tot_pay', 'max_pay_prc',
            'ord_pcs_ttp', 'ord_env_cli_dat', 'tem_de_env_dia', 'flg_atr',
            'ord_apd_dat', 'ord_env_pst_dat', 'ord_est_env_dat',
            'rev_sco', 'rev_cre_dat', 'rev_ans_ttp',
            'cli_zip_cod_prf', 'cli_cit', 'cli_sta', 'geo_lat', 'geo_lng'
        ]

        df_ins = df_pedidos_load[cols_pedidos_load].copy()

        # ---------- TRATAMENTO das DATAS ----------
        dt_cols = [
            'ord_pcs_ttp',
            'ord_env_cli_dat',
            'ord_apd_dat',
            'ord_env_pst_dat',
            'ord_est_env_dat',
            'rev_cre_dat',
            'rev_ans_ttp',
        ]

        for col in dt_cols:
            df_ins[col] = pd.to_datetime(df_ins[col], errors='coerce')

        # ---------- TRATAMENTO das NUMÉRICAS ----------
        num_cols = [
            'qtd_pay_seq', 'val_tot_pay', 'max_pay_prc',
            'tem_de_env_dia', 'flg_atr', 'rev_sco',
            'cli_zip_cod_prf', 'geo_lat', 'geo_lng'
        ]

        for col in num_cols:
            df_ins[col] = pd.to_numeric(df_ins[col], errors='coerce')

        # ---------- TRATAR NaN / NaT → None ----------
        df_ins = df_ins.astype(object)
        df_ins = df_ins.where(df_ins.notna(), None)

        # ---------- GERAR LINHAS SEM .to_numpy ----------
        rows = list(df_ins.itertuples(index=False, name=None))

        # ---------- INSERIR TODOS ----------
        cur.executemany(insert_query, rows)

        # ---------- INSERIR UNKNOWN ----------
        cur.execute(
            """INSERT INTO DW.DIM_ORD (
                rev_com_tit, rev_com_msn, cli_unq_id, ord_stt,
                qtd_pay_seq, pri_pay_typ, val_tot_pay, max_pay_prc,
                ord_pcs_ttp, ord_env_cli_dat, tem_de_env_dia, flg_atr,
                ord_apd_dat, ord_env_pst_dat, ord_est_env_dat,
                rev_sco, rev_cre_dat, rev_ans_ttp,
                cli_zip_cod_prf, cli_cit, cli_sta, geo_lat, geo_lng
            ) VALUES (
                NULL, NULL, 'UNKNOWN', 'unknown',
                NULL, NULL, NULL, NULL,
                '1900-01-01', NULL, NULL, NULL,
                NULL, NULL, NULL,
                NULL, NULL, NULL,
                NULL, NULL, NULL, NULL, NULL
            )
            RETURNING "SRK_ord";"""
        )
        unknown_ord_key = cur.fetchone()[0]

    conn.commit()

print("Chave surrogate do pedido UNKNOWN:", unknown_ord_key)


Chave surrogate do pedido UNKNOWN: 98910


Esta célula faz o mapeamento das chaves substitutas (SRKs) das dimensões para o DataFrame principal.
Ela lê as tabelas dimensionais do banco, realiza os joins com o DataFrame original (df) e substitui valores ausentes pelas chaves 'desconhecidas'.
O resultado é o DataFrame df_fato, já com todas as referências dimensionais resolvidas e pronto para ser inserido na tabela fato FATO_ITENS_PEDIDO.

In [140]:
with connect(connection_string) as conn:
    df_produtos_com_chaves = pd.read_sql('SELECT * FROM DW.DIM_PRD', conn)
    df_vendedores_com_chaves = pd.read_sql('SELECT * FROM DW.DIM_VND', conn)
    df_pedidos_com_chaves = pd.read_sql('SELECT * FROM DW.DIM_ORD', conn)
    df_data_com_chaves = pd.read_sql('SELECT * FROM DW.DIM_DAT', conn)

# Preparar DataFrame para merge
df_m = df.copy()

# Preparar coluna de data para merge
df_m['dat_cmp'] = df_m['ord_pcs_ttp'].dt.date
df_data_com_chaves['dat_cmp'] = pd.to_datetime(df_data_com_chaves['dat_cmp']).dt.date

# Renomear colunas de vendedores para merge
# df_vendedores_com_chaves já tem geo_lat e geo_lng corretos, não precisa renomear

# Renomear colunas de pedidos para merge
df_pedidos_com_chaves = df_pedidos_com_chaves.rename(columns={
    'geo_lat': 'cliente_geo_lat',
    'geo_lng': 'cliente_geo_lng'
})

# Realizar merges
df_m = pd.merge(df_m, df_produtos_com_chaves.drop_duplicates(subset=cols_produtos), on=cols_produtos, how='left')
df_m = pd.merge(df_m, df_vendedores_com_chaves.drop_duplicates(subset=cols_vendedores), on=cols_vendedores, how='left')
df_m = pd.merge(df_m, df_data_com_chaves.drop_duplicates(subset=['dat_cmp']), on='dat_cmp', how='left')

# Preparar colunas de pedidos para merge (sem tem_de_env_dia e flg_atr que são calculados)
cols_pedidos_merge = [c for c in cols_pedidos if c not in ['tem_de_env_dia', 'flg_atr']]
cols_pedidos_merge.extend(['tem_de_env_dia', 'flg_atr'])
df_m = pd.merge(df_m, df_pedidos_com_chaves.drop_duplicates(subset=cols_pedidos_merge), on=cols_pedidos_merge, how='left')

# Preencher chaves ausentes com valores "unknown"
df_m['SRK_prd'] = df_m['SRK_prd'].fillna(unknown_prod_key).astype(int)
df_m['SRK_vnd'] = df_m['SRK_vnd'].fillna(unknown_vend_key).astype(int)
df_m['SRK_dat'] = df_m['SRK_dat'].fillna(unknown_data_key).astype(int)
df_m['SRK_ord'] = df_m['SRK_ord'].fillna(unknown_ord_key).astype(int)

# Selecionar colunas para tabela fato
cols_fato = ['SRK_prd', 'SRK_vnd', 'SRK_dat', 'SRK_ord', 'env_lmt_dat', 'val', 'frt_val']
df_fato = df_m[cols_fato].copy()

Esta célula realiza a carga final da tabela fato FATO_ITENS_PEDIDO no schema DW.
Ela insere todos os registros do DataFrame df_fato, já com as chaves substitutas das dimensões, consolidando os dados no modelo estrela do Data Warehouse.

In [141]:
with connect(connection_string) as conn:
    with conn.cursor() as cur:
        insert_query = sql.SQL(
            """INSERT INTO DW.FAT_ITS_ORD (
                "SRK_prd", "SRK_vnd", "SRK_dat_ord", "SRK_ord",
                env_lmt_dat, val, frt_val
            ) VALUES (%s, %s, %s, %s, %s, %s, %s)"""
        )

        cols = ['SRK_prd', 'SRK_vnd', 'SRK_dat', 'SRK_ord', 'env_lmt_dat', 'val', 'frt_val']

        def _to_db(v):
            if v is None:
                return None
            if isinstance(v, float) and math.isnan(v):
                return None
            if pd.isna(v):
                return None
            return v

        rows = [
            tuple(_to_db(v) for v in row)
            for row in df_fato[cols].itertuples(index=False, name=None)
        ]

        cur.executemany(insert_query, rows)

In [143]:
with connect(connection_string) as conn:
    with conn.cursor() as cur:
        cur.execute('SELECT COUNT(*) FROM DW.DIM_PRD'); dim_produtos_count = cur.fetchone()[0]
        cur.execute('SELECT COUNT(*) FROM DW.DIM_VND'); dim_vendedores_count = cur.fetchone()[0]
        cur.execute('SELECT COUNT(*) FROM DW.DIM_ORD'); dim_pedidos_count = cur.fetchone()[0]
        cur.execute('SELECT COUNT(*) FROM DW.DIM_DAT'); dim_data_count = cur.fetchone()[0]
        cur.execute('SELECT COUNT(*) FROM DW.FAT_ITS_ORD'); fato_count = cur.fetchone()[0]

print(f"Registros carregados no banco:")
print(f"DIM_PRD: {dim_produtos_count}")
print(f"DIM_VND: {dim_vendedores_count}")
print(f"DIM_ORD: {dim_pedidos_count}")
print(f"DIM_DAT: {dim_data_count}")
print(f"FAT_ITS_ORD: {fato_count}")

Registros carregados no banco:
DIM_PRD: 32257
DIM_VND: 2297
DIM_ORD: 98910
DIM_DAT: 617
FAT_ITS_ORD: 112952
